In [1]:
import pandas as pd
import numpy as np
import pickle

import xgboost as xg

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.svm import SVC

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras import utils

In [2]:
pd.set_option("display.max_columns", None)

In [3]:
data = pd.read_csv('../data/chi_df_model_ready.csv')
data.head()

,severity,start_time,end_time,start_lat,start_lng,end_lat,end_lng,distance(mi),number,weather_timestamp,temperature(f),wind_chill(f),humidity(%),pressure(in),visibility(mi),wind_speed(mph),precipitation(in),amenity,bump,crossing,give_way,junction,no_exit,railway,roundabout,station,stop,traffic_calming,traffic_signal,turning_loop,wind_direction_Calm,wind_direction_E,wind_direction_ENE,wind_direction_ESE,wind_direction_East,wind_direction_N,wind_direction_NE,wind_direction_NNE,wind_direction_NNW,wind_direction_NW,wind_direction_North,wind_direction_S,wind_direction_SE,wind_direction_SSE,wind_direction_SSW,wind_direction_SW,wind_direction_South,wind_direction_VAR,wind_direction_Variable,wind_direction_W,wind_direction_WNW,wind_direction_WSW,wind_direction_West,wind_direction_nan,weather_condition_Clear,weather_condition_Cloudy,weather_condition_Cloudy / Windy,weather_condition_Drizzle,weather_condition_Fair,weather_condition_Fair / Windy,weather_condition_Fog,weather_condition_Freezing Rain,weather_condition_Haze,weather_condition_Heavy Drizzle,weather_condition_Heavy Rain,weather_condition_Heavy Rain / Windy,weather_condition_Heavy Snow,weather_condition_Heavy T-Storm,weather_condition_Heavy T-Storm / Windy,weather_condition_Heavy Thunderstorms and Rain,weather_condition_Light Drizzle,weather_condition_Light Drizzle / Windy,weather_condition_Light Freezing Drizzle,weather_condition_Light Freezing Fog,weather_condition_Light Freezing Rain,weather_condition_Light Freezing Rain / Windy,weather_condition_Light Ice Pellets,weather_condition_Light Rain,weather_condition_Light Rain / Windy,weather_condition_Light Rain Showers,weather_condition_Light Rain with Thunder,weather_condition_Light Snow,weather_condition_Light Snow / Windy,weather_condition_Light Snow and Sleet / Windy,weather_condition_Light Thunderstorms and Rain,weather_condition_Light Thunderstorms and Snow,weather_condition_Mist,weather_condition_Mostly Cloudy,weather_condition_Mostly Cloudy / Windy,weather_condition_N/A Precipitation,weather_condition_Overcast,weather_condition_Partly Cloudy,weather_condition_Partly Cloudy / Windy,weather_condition_Patches of Fog,weather_condition_Rain,weather_condition_Rain / Windy,weather_condition_Scattered Clouds,weather_condition_Smoke,weather_condition_Snow,weather_condition_Snow / Windy,weather_condition_Squalls,weather_condition_Squalls / Windy,weather_condition_T-Storm,weather_condition_T-Storm / Windy,weather_condition_Thunder,weather_condition_Thunder in the Vicinity,weather_condition_Thunderstorm,weather_condition_Thunderstorms and Rain,weather_condition_Wintry Mix,weather_condition_Wintry Mix / Windy,weather_condition_nan,sunrise_sunset_Night,civil_twilight_Night,nautical_twilight_Night,astronomical_twilight_Night,accident_duration
0,2,1.480517e+09,1.480539e+09,42.305960,-87.960150,42.306916,-87.960918,0.077,0.0,1.480518e+09,37.0,31.0,79.0,29.59,10.0,8.1,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21600.0
1,2,1.480518e+09,1.480540e+09,42.175870,-88.135770,42.174620,-88.135794,0.086,21500.0,1.480518e+09,37.0,28.1,76.0,29.62,10.0,15.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21600.0
2,3,1.480519e+09,1.480541e+09,42.175897,-88.135769,42.174612,-88.135794,0.089,21501.0,1.480518e+09,37.0,28.1,76.0,29.62,10.0,15.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22524.0
3,3,1.480522e+09,1.480544e+09,41.860591,-87.992749,41.860590,-87.992700,0.003,1194.0,1.480522e+09,37.0,30.4,76.0,29.61,10.0,9.2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0

In [4]:
X = data.drop(['severity','start_time','end_time','start_lng','end_lat','weather_timestamp','accident_duration'], axis=1)
y = data['severity']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [5]:
print('X_train shape:',X_train.shape)
print('y_train shape:',y_train.shape)
print('X_test shape:',X_test.shape)
print('y_test shape:',y_test.shape)

X_train shape: (31805, 109)
y_train shape: (31805,)
X_test shape: (10602, 109)
y_test shape: (10602,)


In [6]:
y.value_counts(normalize=True)

3    0.498998
2    0.422100
4    0.073030
1    0.005872
Name: severity, dtype: float64

In [7]:
# # initialize a base logistic regression and fit for multinomial classification
# logreg = LogisticRegression(max_iter=10_000, multi_class='multinomial', solver='lbfgs')
# logreg.fit(X_train,y_train)

In [8]:
# # print out our accuracy scores
# print('Logistic Regression Training Accuracy:', logreg.score(X_train,y_train))
# print('Logistic Regression Testing Accuracy:', logreg.score(X_test,y_test))

In [9]:
xg_model = xg.sklearn.XGBClassifier()
xg_model.fit(X_train,y_train);

[20:13:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [10]:
# print out our accuracy scores
print('XGBoost Training Accuracy:', xg_model.score(X_train,y_train))
print('XGBoost Testing Accuracy:', xg_model.score(X_test,y_test))

XGBoost Training Accuracy: 0.825719226536708
XGBoost Testing Accuracy: 0.7219392567440106


In [12]:
#save the model
with open('./models/xgboost.pkl', mode='wb') as pickle_out:
    pickle.dump(xg_model, pickle_out)

In [13]:
# decision tree model with just a few hyperparameters
tree = DecisionTreeClassifier(max_depth=8, min_samples_split=5)
tree.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=8, min_samples_split=5)

In [14]:
print('Decision Tree Training Accuracy:', tree.score(X_train,y_train))
print('Decision Tree Testing Accuracy:', tree.score(X_test,y_test))

Decision Tree Training Accuracy: 0.6832887910705864
Decision Tree Testing Accuracy: 0.6506319562346727


In [15]:
forrest = RandomForestClassifier(max_depth=10)
forrest.fit(X_train,y_train)

RandomForestClassifier(max_depth=10)

In [16]:
print('Random Forrest Training Accuracy:', forrest.score(X_train,y_train))
print('Random Forrest Testing Accuracy:', forrest.score(X_test,y_test))

Random Forrest Training Accuracy: 0.6919352303096997
Random Forrest Testing Accuracy: 0.6562912657989058


In [17]:
bag = BaggingClassifier()
bag.fit(X_train,y_train)

BaggingClassifier()

In [18]:
print('BAG Training Accuracy:', bag.score(X_train,y_train))
print('BAG Testing Accuracy:', bag.score(X_test,y_test))

BAG Training Accuracy: 0.9853167740921239
BAG Testing Accuracy: 0.7237313714393511


In [19]:
with open('./models/BAG.pkl', mode='wb') as pickle_out:
    pickle.dump(bag, pickle_out)

In [20]:
# supvec = SVC()
# supvec.fit(X_train,y_train)

In [21]:
# print('SVM Training Accuracy:', supvec.score(X_train,y_train))
# print('SVM Testing Accuracy:', supvec.score(X_test,y_test))

In [22]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

In [23]:
y_train_cat = utils.to_categorical(y_train-1, 4)
y_test_cat = utils.to_categorical(y_test-1, 4)

In [24]:
y_test_cat

array([[0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]], dtype=float32)

In [25]:
nn = Sequential()

nn.add(
    Dense(
        128,
        activation = 'relu'
    )
)

nn.add(Dropout(.4))

nn.add(
    Dense(
        64,
        activation = 'relu'
    )
)

nn.add(Dropout(.3))

nn.add(
    Dense(
        32,
        activation = 'relu'
    )
)

nn.add(Dropout(.2))

nn.add(
    Dense(
        16,
        activation = 'relu'
    )
)

nn.add(Dropout(.1))

nn.add(
    Dense(
        8,
        activation = 'relu'
    )
)

nn.add(
    Dense(
        4,
        activation = 'softmax'
    )
)

nn.compile(
    loss='categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [26]:
history = nn.fit(
    X_train_sc, 
    y_train_cat,
    validation_data=(X_test_sc, y_test_cat),
    epochs = 256,
    verbose = 0
)

In [27]:
max(history.history['accuracy']),max(history.history['val_accuracy'])

(0.725640594959259, 0.693265438079834)

In [28]:
nn.save('./models/neural_net')

INFO:tensorflow:Assets written to: ./models/neural_net\assets
